In [4]:
import cv2
import os
import urllib.request

# Download Haar Cascade if not present
haar_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
haar_path = "haarcascade_frontalface_default.xml"

if not os.path.exists(haar_path):
    print("Downloading Haar Cascade...")
    urllib.request.urlretrieve(haar_url, haar_path)
    print("Download complete.")

# Initialize the Haar Cascade classifier
face_cascade = cv2.CascadeClassifier(haar_path)

# Initialize camera
cap = cv2.VideoCapture(0)  # 0 for default camera

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(60, 60),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw rectangles around faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face Detection', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import os
import numpy as np

# Paths to the models
FACE_DETECTION_MODEL = "models/res10_300x300_ssd_iter_140000_fp16.caffemodel"
FACE_DETECTION_PROTO = "models/deploy.prototxt"
FACE_EMBEDDING_MODEL = "models/openface_nn4.small2.v1.t7"

# Thresholds
CONFIDENCE_THRESHOLD = 0.5

# Initialize the face detector
face_net = cv2.dnn.readNetFromCaffe(FACE_DETECTION_PROTO, FACE_DETECTION_MODEL)

# Initialize the face embedding model
embed_net = cv2.dnn.readNetFromTorch(FACE_EMBEDDING_MODEL)

# Initialize camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

def extract_features():
    user_name = input("Enter the name of the authorized user: ")
    dataset_path = f"dataset/{user_name}"
    os.makedirs(dataset_path, exist_ok=True)

    count = 0
    TOTAL_IMAGES = 20

    while count < TOTAL_IMAGES:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Resize frame to have a width of 600 pixels for faster processing
        frame = cv2.resize(frame, (600, int(frame.shape[0] * 600 / frame.shape[1])))

        # Convert the frame to a blob for face detection
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                                     (300, 300), (104.0, 177.0, 123.0))

        # Detect faces in the frame
        face_net.setInput(blob)
        detections = face_net.forward()

        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            if confidence < CONFIDENCE_THRESHOLD:
                continue

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Ensure the bounding boxes fall within the dimensions of the frame
            startX = max(0, startX)
            startY = max(0, startY)
            endX = min(w, endX)
            endY = min(h, endY)

            # Extract the face ROI
            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            # Ensure the face is large enough
            if fW < 20 or fH < 20:
                continue

            # Construct a blob for the face ROI, then pass it through the embedding model
            face_blob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                                              (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embed_net.setInput(face_blob)
            vec = embed_net.forward()

            # Save the face image and its embedding
            face_path = os.path.join(dataset_path, f"img_{count}.jpg")
            cv2.imwrite(face_path, face)
            np.save(os.path.join(dataset_path, f"img_{count}.npy"), vec)

            print(f"Captured image {count+1}/{TOTAL_IMAGES}")
            count += 1

            # Draw a rectangle around the face and display
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                          (0, 255, 0), 2)
            cv2.putText(frame, f"Image {count}/{TOTAL_IMAGES}", (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            if count >= TOTAL_IMAGES:
                break

        # Display the frame
        cv2.imshow('Feature Extraction - Press Q to Quit', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    extract_features()
